In [62]:
import pandas as pd
from collections import deque

In [53]:
plays = pd.read_csv('../data/play_by_play.txt', delimiter = "\t")
lineup = pd.read_csv('../data/lineup.txt', delimiter = "\t").sort_values(['Game_id', 'Period', 'Team_id'], 
                                                                         ascending = [True, True, True])
plays = plays.sort_values(['Game_id', 'Period', 'PC_Time', 'WC_Time', 'Event_Num'], 
                 axis = 0, ascending = [True, True, False, True, True]).drop(['Team_id_type', 'Option2', 'Option3'], axis = 1)
game_ids = plays['Game_id'].unique().tolist()
lineup0 = lineup[lineup.Game_id == game_ids[0]].reset_index()
df = plays[plays.Game_id == game_ids[0]]
# something with views if error in future

In [68]:
team1players = {}
team2players = {}
t1Active = []
t2Active = []
team1 = ''
team2 = ''
bigList1 = []
bigList2 = []
for i, row in lineup0.iterrows():
    if i < 10:
        if i < 5:
            team1players[row['Person_id']] = 0
            t1Active.append(row['Person_id'])
            team1 = row["Team_id"]
        else:
            team2players[row['Person_id']] = 0
            t2Active.append(row['Person_id'])
            team2 = row["Team_id"]
bigList1.append(t1Active)
bigList2.append(t2Active)
print(t1Active)
print(t2Active)
subsInFoul = deque()
flag = False
for i, event in df.iterrows():
    event_type = event['Event_Msg_Type']
    if event_type == 6:
        flag = True
    if event_type == 8:
        if not flag:
            person1 = event['Person1']
            person2 = event['Person2']
            print(event['Team_id'])
            print(team2)
            if event['Team_id'] == team1:
                t1Active.remove(person1)
                t1Active.append(person2)
                if person2 not in team1players:
                    team1players[person2] = 0
            else:
                print(person1)
                print(t2Active)
                t2Active.remove(person1)
                t2Active.append(person2)
                if person2 not in team2players:
                    team2players[person2] = 0
        else:
            person1 = event['Person1']
            person2 = event['Person2']
            subsInFoul.append((person1, person2, event['Team_id']))
    if flag == True and (event_type == 1 or event_type == 2 or event_type == 4 or event_type == 5 or event_type == 10):
        flag = False
        while len(subsInFoul) != 0:
            sub = subsInFoul.popleft()
            if sub[2] == team1:
                t1Active.remove(sub[0])
                t1Active.append(sub[1])
                if sub[1] not in team1players:
                    team1players[sub[1]] = 0
            else:
                t2Active.remove(sub[0])
                t2Active.append(sub[1])
                if sub[1] not in team2players:
                    team2players[sub[1]] = 0
    bigList1.append(t1Active)
    bigList2.append(t2Active)

['881f83d2dee3f18c7d1751659406144e', 'cec898a1d355dbfbad8c760615fde1af', 'a99f44bbff39e352191a870e17f04537', '89706b99ddd00dc05d37ef5cafc04276', '2b313e2bcef0268bc8e9415132ba9997']
['27ea17a8685c4919f157e83fe9cb2d9e', '57bbd7e30bc694aeee9ee40c583e6811', '33963fe856a1523ff46438ba07d1d99f', 'c00264c3114d23bac482e9de50fb7d28', '307beab25b1021a548b4a47550bc4b25']
cff694c8186a4bd377de400e4f60fe47
cff694c8186a4bd377de400e4f60fe47
89706b99ddd00dc05d37ef5cafc04276
['27ea17a8685c4919f157e83fe9cb2d9e', '57bbd7e30bc694aeee9ee40c583e6811', '33963fe856a1523ff46438ba07d1d99f', 'c00264c3114d23bac482e9de50fb7d28', '307beab25b1021a548b4a47550bc4b25']


ValueError: list.remove(x): x not in list